In [2]:
START_YEAR = 2005
CURRENT_SEASON = 2024
ONLY_CURRENT_SEASON = False
SPORT_IDS = [1,11,12,13,14,15,16]

Get FangraphsIds From Chadwick Register

In [3]:
FILE_NAME = "https://github.com/chadwickbureau/register/raw/master/data/people-"
FILE_EXT = ".csv"
FILE_NUM = [0,1,2,3,4,5,6,7,8,9,"a","b","c","d","e","f"]

In [3]:
import requests
import sqlite3
import pandas as pd
import io
from tqdm import tqdm

In [4]:
db = sqlite3.connect("BaseballStats.db")

In [5]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")

for num in tqdm(FILE_NUM):
    file = FILE_NAME + str(num) + FILE_EXT
    response = requests.get(file)
    if response.status_code != 200:
        print(f"Failed to Get {file} : {response.status_code}")
        continue
    
    reqData = response.content
    df = pd.read_csv(io.StringIO(reqData.decode('utf-8')), on_bad_lines="skip")
    df = df[df['key_fangraphs'].notna()][df['key_mlbam'].notna()]
    df['key_mlbam'] = df['key_mlbam'].astype('Int64')
    df['key_fangraphs'] = df['key_fangraphs'].astype('Int64')
    for row in df.itertuples():
        cursor.execute(f"UPDATE Player Set fangraphsId='{row.key_fangraphs}' WHERE mlbId='{row.key_mlbam}'")
    
cursor.execute("END TRANSACTION")
db.commit()

  0%|          | 0/16 [00:00<?, ?it/s]C:\Users\nitzr\AppData\Local\Temp\ipykernel_6624\1149116774.py:13: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(reqData.decode('utf-8')), on_bad_lines="skip")
C:\Users\nitzr\AppData\Local\Temp\ipykernel_6624\1149116774.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df['key_fangraphs'].notna()][df['key_mlbam'].notna()]
  6%|▋         | 1/16 [00:03<00:49,  3.31s/it]C:\Users\nitzr\AppData\Local\Temp\ipykernel_6624\1149116774.py:13: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(reqData.decode('utf-8')), on_bad_lines="skip")
C:\Users\nitzr\AppData\Local\Temp\ipykernel_6624\1149116774.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df['key_fangraphs'].notna()][df['key_mlbam'].notna()]
 12%|█▎     

Get War From Fangraphs

In [6]:
import pybaseball

In [7]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")

for year in tqdm(range(START_YEAR, CURRENT_SEASON)):
    if ONLY_CURRENT_SEASON and year != CURRENT_SEASON:
        continue
    
    # For some reason no OPS breaks it
    hittingStats = pybaseball.batting_stats(year, qual=0, stat_columns=["PA", "BsR", "Off", "Def", "WAR", "OPS"])
    for row in hittingStats.itertuples():
        mlbId = cursor.execute("SELECT mlbId FROM Player WHERE fangraphsId=?", (row.IDfg,)).fetchone()[0]
        if mlbId == None or cursor.execute("SELECT COUNT(*) FROM Player_YearlyWar WHERE mlbId=? AND year=? AND position=?", (mlbId, year, "hitting")).fetchone()[0] > 0:
            continue
        
        cursor.execute("INSERT INTO Player_YearlyWar VALUES(?,?,?,?,?,?,?,?)", (mlbId, year, "hitting", row.PA, row.WAR, row.Off, row.Def, row.BsR))
    
    
    # Similar to above, W is needed
    pitchingStats = pybaseball.pitching_stats(year, qual=0, stat_columns = ["IP", "WAR", "W"])
    for row in pitchingStats.itertuples():
        mlbId = cursor.execute("SELECT mlbId FROM Player WHERE fangraphsId=?", (row.IDfg,)).fetchone()[0]
        if mlbId == None or cursor.execute("SELECT COUNT(*) FROM Player_YearlyWar WHERE mlbId=? AND year=? and position=?", (mlbId, year, "pitching")).fetchone()[0] > 0:
            continue
        
        innings, subinnings = str(row.IP).split('.')
        outs = 3 * innings + subinnings
        
        cursor.execute("INSERT INTO Player_YearlyWar VALUES(?,?,?,?,?,?,?,?)", (mlbId, year, "pitching", outs, row.WAR, 0, 0, 0))
        
cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 19/19 [01:49<00:00,  5.76s/it]


Update the status of player careers

In [8]:
UPDATE_YEAR = 2023

In [9]:
NUM_EMPTY_YEARS = 2 # Players who have not played in last N years are considered retired
SERVICE_TIME_CUTOFF = 6 # Players who have reached this service time have reached FA and no longer accrue prospect value
AGED_OUT_AGE = 27 # If a player has not made the MLB by this age, the prospect value becomes 
AGED_OUT_MONTH = 4
ROOKIE_CUTTOFF_DAYS = 60

Insert any players that do not exist

In [10]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")
idDatas = cursor.execute("SELECT mlbId, position FROM Player ORDER BY mlbId DESC").fetchall()
for id, position in tqdm(idDatas):
    if cursor.execute("SELECT COUNT(*) FROM Player_CareerStatus WHERE mlbId=? AND position=?", (id, position)).fetchone()[0] > 0:
        continue
    
    cursor.execute("INSERT INTO Player_CareerStatus('mlbId','position') VALUES(?,?)", (id, position))
    
cursor.execute("END TRANSACTION")
db.commit()

  0%|          | 0/52095 [00:00<?, ?it/s]

100%|██████████| 52095/52095 [00:00<00:00, 314405.32it/s]


Update primary position for players who don't have one

In [11]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")
unknownPositions = cursor.execute("SELECT mlbId, position FROM Player_CareerStatus WHERE isPrimaryPosition IS NULL").fetchall()
for id, position in tqdm(unknownPositions):
    sumBF = cursor.execute("SELECT SUM(battersFaced) FROM Player_Pitcher_MonthStats WHERE mlbId=?", (id,)).fetchone()[0]
    sumPA = cursor.execute("SELECT SUM(AB) FROM Player_Hitter_MonthStats WHERE mlbId=?", (id,)).fetchone()[0]
    
    # If not found at all, need to set to zero
    if sumBF == None:
        sumBF = 0
    if sumPA == None:
        sumPA = 0

    # Set to whichever position has more
    # Only player this doesn't work for is Ohtani, who will be excluded for being a NPB signee
    if (sumBF == 0) and (sumPA == 0):
        continue
    isHitter = sumPA > sumBF
    if (isHitter and position=="hitting") or (not isHitter and position=="pitching"):
        isPrimary = 1
    else:
        isPrimary = 0
        
    cursor.execute("UPDATE Player_CareerStatus SET isPrimaryPosition=? WHERE mlbId=? AND position=?", (isPrimary, id, position))
    
cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 52095/52095 [00:01<00:00, 42877.62it/s]


Check if player is currently active

In [12]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")
idDatas = cursor.execute("SELECT mlbId, position FROM Player_CareerStatus WHERE isPrimaryPosition='1'").fetchall()
for id, position in tqdm(idDatas):
    if position == "hitting":
        table = "Player_Hitter_MonthStats"
    else:
        table = "Player_Pitcher_MonthStats"
        
    lastYear = cursor.execute(f"SELECT Year FROM {table} WHERE mlbId=? ORDER BY Year DESC LIMIT 1", (id,)).fetchone()[0]
    if (UPDATE_YEAR - lastYear) >= 2:
        isActive = 0
    else:
        isActive = 1
        
    cursor.execute("UPDATE Player_CareerStatus SET isActive=? WHERE mlbId=?", (isActive, id))
    
cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 42524/42524 [00:00<00:00, 229937.76it/s]


Check if player has had their service time reached

In [13]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")
for id, position in tqdm(idDatas):
    serviceYears = cursor.execute("SELECT serviceYear FROM Player_ServiceTime WHERE mlbId=? ORDER BY serviceYear DESC", (id,)).fetchone()
    if serviceYears == None:
        serviceReached = 0
    elif serviceYears[0] < SERVICE_TIME_CUTOFF:
        serviceReached = 0
    else:
        serviceReached = 1
        
    cursor.execute("UPDATE Player_CareerStatus SET serviceReached=? WHERE mlbId=?", (serviceReached, id))

cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 42524/42524 [00:00<00:00, 265810.90it/s]


Set MLB Start Year

In [14]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")
for id, _ in tqdm(idDatas, desc="MLB Start Year Check"):
    pitchingYear = cursor.execute("SELECT year FROM Player_Pitcher_MonthStats WHERE mlbId=? AND level=? ORDER BY year ASC LIMIT 1", (id, 1)).fetchone()
    hittingYear = cursor.execute("SELECT year FROM Player_Hitter_MonthStats WHERE mlbId=? AND levelId=? ORDER BY year ASC LIMIT 1", (id, 1)).fetchone()
    if pitchingYear == None:
        if hittingYear == None: #Never made mlb
            continue
        
        # only made as hitter
        startYear = hittingYear[0]
    else:
        if hittingYear == None: # only made as pitcher
            startYear = pitchingYear[0]
        else: # made as both
            if hittingYear[0] < pitchingYear[0]:
                startYear = hittingYear[0]
            else:
                startYear = pitchingYear[0]
            
    
    #startYear = cursor.execute("SELECT year FROM Player_YearlyWar WHERE mlbId=? ORDER BY year ASC", (id,)).fetchone()
    if startYear is not None:
        cursor.execute("UPDATE Player_CareerStatus SET mlbStartYear=? WHERE mlbId=?", (startYear, id))

cursor.execute("END TRANSACTION")
db.commit()

MLB Start Year Check: 100%|██████████| 42524/42524 [00:00<00:00, 307063.20it/s]


Set Service End Year

In [15]:
db.rollback()
cursor = db.cursor()
mlbIdDatas = cursor.execute("SELECT mlbId, position FROM Player_CareerStatus WHERE mlbStartYear IS NOT NULL").fetchall()
cursor.execute("BEGIN TRANSACTION")
for id, position in tqdm(idDatas):
    serviceData = cursor.execute("SELECT year, serviceYear, serviceDays FROM Player_ServiceTime WHERE mlbId=? ORDER BY year DESC", (id,)).fetchall()
    if serviceData == None or len(serviceData) == 0:
        continue
    
    if serviceData[0][1] < 6: # Service time has not been reached
        continue 
    
    for year, sy, sd in serviceData:
        if sy < SERVICE_TIME_CUTOFF or (sy == SERVICE_TIME_CUTOFF and sd == 0):
            cursor.execute("UPDATE Player_CareerStatus SET serviceEndYear=? WHERE mlbId=?", (year, id))
            break
        
cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 42524/42524 [00:00<00:00, 635775.61it/s]


Get first year of career

Need to check if any pre-05 data exists to exclude

In [18]:
db.rollback()
cursor = db.cursor()
file = open("errors/error.txt", "w")

noCareerStartData = cursor.execute("SELECT mlbId FROM Player_CareerStatus WHERE careerStartYear IS NULL ORDER BY mlbId ASC").fetchall()
for i, id in enumerate(tqdm(noCareerStartData)):
    id = id[0]
    response = requests.get(f"https://statsapi.mlb.com/api/v1/people/{id}?hydrate=currentTeam,team,stats(type=[yearByYear](team(league)),leagueListId=mlb_milb)&site=en")
    if response.status_code != 200:
        print(f"Error for id={id}: {response.status_code}")
        continue
    
    jsonData = response.json()
    try:
        year = jsonData["people"][0]["stats"][0]["splits"][0]["season"]
        cursor.execute("UPDATE Player_CareerStatus SET careerStartYear=? WHERE mlbId=?", (year, id))
    except Exception as e:
        file.write(f"{id}\n")
        
    if i % 100 == 0:
        db.commit()
        
file.close()
db.commit()

  3%|▎         | 1/33 [00:00<00:10,  3.11it/s]

Error for id=0: 404


100%|██████████| 33/33 [00:12<00:00,  2.55it/s]


Use Selenium to get the players from error.txt to manually populate

In [37]:
from selenium import webdriver

In [ ]:
db.rollback()
cursor = db.cursor()

file = open("errors/error.txt", "r")
driver = webdriver.Chrome()

for id in file:
    driver.get(f"https://www.milb.com/player/{id}")
    year = input("Start Year")
    cursor.execute("UPDATE Player_CareerStatus SET careerStartYear=? WHERE mlbId=?", (year, id))
    db.commit()
    cursor = db.cursor()


Check if player did not reach MLB before certain age

First, check players who have not made MLB

In [19]:
db.rollback()
cursor = db.cursor()

noMlbPlayers = cursor.execute("SELECT mlbId FROM Player_CareerStatus WHERE mlbStartYear IS NULL").fetchall()
cursor.execute("BEGIN TRANSACTION")
cutoffYear = CURRENT_SEASON + 1 - AGED_OUT_AGE
for id in tqdm(noMlbPlayers):
    id = id[0]
    birthYear, birthMonth = cursor.execute("SELECT birthYear, birthMonth FROM Player WHERE mlbId=?", (id,)).fetchone()
    
    # Need to eliminate any players who don't have a listed birth date
    if birthYear == None or birthMonth == None:
        cursor.execute("UPDATE Player_CareerStatus SET agedOut=?, ignorePlayer=? WHERE mlbId=?", (1, 1, id))
        continue
    
    if birthMonth >= 4:
        birthYear += 1
    if birthYear < cutoffYear:
        agedOut = birthYear + AGED_OUT_AGE
    else:
        agedOut = 0
        
    cursor.execute("UPDATE Player_CareerStatus SET agedOut=? WHERE mlbId=?", (agedOut, id))

cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 42714/42714 [00:00<00:00, 240010.05it/s]


Next, check that players who made the MLB didn't have their first year after 27

In [20]:
db.rollback()
cursor = db.cursor()

noMlbPlayers = cursor.execute("SELECT mlbId, mlbStartYear FROM Player_CareerStatus WHERE mlbStartYear IS NOT NULL").fetchall()
cursor.execute("BEGIN TRANSACTION")
for id, startYear in tqdm(noMlbPlayers):
    birthYear, birthMonth = cursor.execute("SELECT birthYear, birthMonth FROM Player WHERE mlbId=?", (id,)).fetchone()
    if birthMonth >= 4:
        birthYear += 1
    
    if startYear - birthYear > AGED_OUT_AGE:
        agedOut = birthYear + AGED_OUT_AGE
    else:
        agedOut = 0
        
    cursor.execute("UPDATE Player_CareerStatus SET agedOut=? WHERE mlbId=?", (agedOut, id))

cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 9381/9381 [00:00<00:00, 193220.12it/s]


Check to see when a player reached rookie eligibility (60 service days)

After this point, the player is no longer a prospect by the model

In [26]:
db.rollback()
cursor = db.cursor()
mlbIdDatas = cursor.execute("SELECT mlbId FROM Player_CareerStatus WHERE mlbStartYear IS NOT NULL").fetchall()
cursor.execute("BEGIN TRANSACTION")
for id in tqdm(mlbIdDatas):
    id = id[0]
    serviceData = cursor.execute("SELECT year, serviceYear, serviceDays FROM Player_ServiceTime WHERE mlbId=? ORDER BY year ASC", (id,)).fetchall()
    if serviceData == None or len(serviceData) == 0:
        continue
    
    # Check that the last element (first entry) has service time <= 1 year.  Don't have full data
    if serviceData[0][1] > 1 or (serviceData[0][1] == 1 and serviceData[0][2] > 0):
        continue
    
    for year, sy, sd in serviceData:
        if sy > 0 or sd >= SERVICE_TIME_CUTOFF:
            cursor.execute("UPDATE Player_CareerStatus SET mlbRookieYear=? WHERE mlbId=?", (year, id))
            break
        
cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 9381/9381 [00:00<00:00, 212960.34it/s]


Last column is ignorePlayer.  This is intended for players that the model doesn't have good data to predict (eg players from Japan/Korea that go directly to the MLB)

The file with the player list will be edited manually

In [22]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")
cursor.execute("UPDATE Player_CareerStatus SET ignorePlayer=NULL")

file = open("ignorePlayers.txt", "r")
for id in file:
    cursor.execute("UPDATE Player_CareerStatus SET ignorePlayer=? WHERE mlbId=?", (1, id))

cursor.execute("END TRANSACTION")
db.commit()

Set the highest level the player has reached

In [33]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")
for id, _ in tqdm(idDatas):
    # Determine if age cuttoff is needed
    cutoffYear = cursor.execute("SELECT agedOut FROM Player_CareerStatus WHERE mlbId=?", (id,)).fetchone()[0]
    if cutoffYear == 0:
        cutoffYear = 9000
    
    pitchingLevel = cursor.execute("SELECT Level FROM Player_Pitcher_MonthStats WHERE mlbId=? AND Year<=? ORDER BY Level ASC LIMIT 1", (id, cutoffYear)).fetchone()
    hittingLevel = cursor.execute("SELECT LevelId FROM Player_Hitter_MonthStats WHERE mlbId=? AND Year<=? ORDER BY LevelId ASC LIMIT 1", (id, cutoffYear)).fetchone()
    
    if pitchingLevel == None:
        if hittingLevel == None: 
            continue
        
        # Only hitting stats
        highestLevel = hittingLevel[0]
    else:
        if hittingLevel == None: # Only pitching stats
            highestLevel = pitchingLevel[0]
        else: # both stats, choose highest level
            if pitchingLevel[0] < pitchingLevel[0]:
                highestLevel = pitchingLevel[0]
            else:
                highestLevel = hittingLevel[0]
            
    
    #startYear = cursor.execute("SELECT year FROM Player_YearlyWar WHERE mlbId=? ORDER BY year ASC", (id,)).fetchone()
    if highestLevel is not None:
        cursor.execute("UPDATE Player_CareerStatus SET highestLevel=? WHERE mlbId=?", (highestLevel, id))

cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 42524/42524 [00:00<00:00, 86337.66it/s]


Mising some service time data.  First check is just to find players that, in their debut year, had >50 innings or >130AB to set their rookie year

In [41]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")

ids = cursor.execute('''
                     SELECT pcs.mlbID, pcs.position, pcs.mlbStartYear FROM Player_CareerStatus AS pcs 
                     INNER JOIN Player_ServiceTime AS pst ON pst.mlbId = pcs.mlbId 
                     AND pcs.mlbRookieYear IS NULL
                     AND pcs.agedOut='0'
                     AND pcs.ignorePlayer IS NULL
                     AND pcs.careerStartYear>='2005'
                     AND pcs.isPrimaryPosition='1'
                     ''').fetchall()

for id, position, year in tqdm(ids):
    if position == 'pitching':
        outs = cursor.execute("SELECT SUM(outs) FROM Player_Pitcher_MonthStats WHERE mlbId=? AND year=? AND level='1'", (id, year)).fetchone()[0]
        if outs is not None and outs >= 150:
            cursor.execute("UPDATE Player_CareerStatus SET mlbRookieYear=? WHERE mlbId=?", (year, id))
    
    else: # hitting
        ab = cursor.execute("SELECT SUM(AB) FROM Player_Hitter_MonthStats WHERE mlbId=? AND year=? AND levelId='1'", (id, year)).fetchone()[0]
        if ab is not None and ab >= 130:
            cursor.execute("UPDATE Player_CareerStatus SET mlbRookieYear=? WHERE mlbId=?", (year, id))
    
cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 3746/3746 [00:01<00:00, 2712.84it/s]


Missing some service time data.  Need to manually determine when a player had their rookie year

In [48]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")

# First pass is players that have 150 career outs.  Determine what year they reached that milestone
ids = cursor.execute('''
                     SELECT pcs.mlbId, SUM(pms.outs) FROM Player_CareerStatus AS pcs
                     INNER JOIN Player_Pitcher_MonthStats AS pms ON pms.mlbId = pcs.mlbId
                     WHERE pms.level='1'
                     AND pcs.mlbRookieYear IS NULL
                     AND pcs.agedOut='0'
                     AND pcs.ignorePlayer IS NULL
                     AND pcs.careerStartYear>='2005'
                     GROUP BY pcs.mlbId
                     ''').fetchall()

for id, outs in tqdm(ids):
    if outs < 150:
        continue
    
    yearlyOuts = cursor.execute("SELECT outs, year FROM Player_Pitcher_MonthStats WHERE mlbId=? AND level=? ORDER BY year ASC", (id,1)).fetchall()
    cumOuts = 0
    for outs, year in yearlyOuts:
        if cumOuts < 150:
            cumOuts += outs
            if cumOuts >= 150:
                cursor.execute("UPDATE Player_CareerStatus SET mlbRookieYear=? WHERE mlbId=?", (year, id))
            
cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 465/465 [00:03<00:00, 134.55it/s]


In [50]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")

# Second pass is players that have 130 career ab.  Determine what year they reached that milestone
ids = cursor.execute('''
                     SELECT pcs.mlbId, SUM(hms.ab) FROM Player_CareerStatus AS pcs
                     INNER JOIN Player_Hitter_MonthStats AS hms ON hms.mlbId = pcs.mlbId
                     WHERE hms.levelId='1'
                     AND pcs.mlbRookieYear IS NULL
                     AND pcs.agedOut='0'
                     AND pcs.ignorePlayer IS NULL
                     AND pcs.careerStartYear>='2005'
                     GROUP BY pcs.mlbId
                     ''').fetchall()

for id, ab in tqdm(ids):
    if ab < 130:
        continue
    
    yearlyOuts = cursor.execute("SELECT outs, year FROM Player_Pitcher_MonthStats WHERE mlbId=? AND level=? ORDER BY year ASC", (id,1)).fetchall()
    cumAb = 0
    for ab, year in yearlyOuts:
        if cumAb < 130:
            cumAb += ab
            if cumAb >= 130:
                cursor.execute("UPDATE Player_CareerStatus SET mlbRookieYear=? WHERE mlbId=?", (year, id))
            
cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 356/356 [00:02<00:00, 144.23it/s]


In [8]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")

# Third pass is players that have reached rookie service time, but not had enough ab/outs
# Choose last mlb year as rookie year
ids = cursor.execute('''
                     SELECT DISTINCT pcs.mlbID FROM Player_CareerStatus AS pcs 
                     INNER JOIN Player_ServiceTime AS pst ON pst.mlbId = pcs.mlbId 
                     WHERE (pst.serviceYear>='1' OR pst.serviceDays>='60')
                     AND pcs.mlbRookieYear IS NULL
                     AND pcs.agedOut='0'
                     AND pcs.ignorePlayer IS NULL
                     AND pcs.careerStartYear>='2005'
                     ''').fetchall()

for (id,) in tqdm(ids):
    lastYear = cursor.execute("SELECT year FROM Player_ServiceTime WHERE mlbId=? ORDER BY year DESC LIMIT 1", (id,)).fetchone()[0]
    if lastYear is not None:
        cursor.execute("UPDATE Player_CareerStatus SET mlbRookieYear=? WHERE mlbId=?", (lastYear, id))
        
cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 173/173 [00:00<00:00, 172971.30it/s]


Set the month that rookie status was exhausted

If refactored, this should be in same table

In [10]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")
cursor.execute("DELETE FROM Player_RookieEligibility")
idData = cursor.execute("SELECT mlbId, position, mlbRookieYear FROM Player_CareerStatus WHERE isPrimaryPosition='1' AND mlbRookieYear IS NOT NULL").fetchall()

for id, position, rookieYear in tqdm(idData):
    if position == 'hitting':
        monthData = cursor.execute("SELECT year, month, AB FROM Player_Hitter_MonthStats WHERE mlbId=? AND LevelId=?", (id, 1)).fetchall()
        cumAb = 0
        rYear = 0
        rMonth = 0
        for year, month, ab in monthData:
            cumAb += ab
            if cumAb > 130:
                rYear = year
                rMonth = month
                break
            
        # Check if the year is the same
        if rYear == rookieYear:
            cursor.execute("INSERT INTO Player_RookieEligibility VALUES(?,?,?)", (id, rYear, rMonth))
        else: # The years don't match, so set the eligibility to the end of the year that Player_CareerStatus has their rookie year
            cursor.execute("INSERT INTO Player_RookieEligibility VALUES(?,?,?)", (id, rookieYear, 13))
    else:
        monthData = cursor.execute("SELECT year, month, outs FROM Player_Pitcher_MonthStats WHERE mlbId=? AND Level=?", (id, 1)).fetchall()
        cumPa = 0
        rYear = 0
        rMonth = 0
        for year, month, pa in monthData:
            cumPa += pa
            if cumPa > 150:
                rYear = year
                rMonth = month
                break
            
        # Check if the year is the same
        if rYear == rookieYear:
            cursor.execute("INSERT INTO Player_RookieEligibility VALUES(?,?,?)", (id, rYear, rMonth))
        else: # The years don't match, so set the eligibility to the end of the year that Player_CareerStatus has their rookie year
            cursor.execute("INSERT INTO Player_RookieEligibility VALUES(?,?,?)", (id, rookieYear, 13))

cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 3634/3634 [00:29<00:00, 121.29it/s]


Add a check for an MLB player that has not played in the MLB in 2 years

Also check for more than 9 years of MLB career

Also check for year where player was over 34

After this, player is likely not accruing value to the team that had him as a prospect

In [31]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")
cursor.execute("DELETE FROM Player_ServiceLapse")

ids = cursor.execute('''SELECT DISTINCT(pcs.mlbId), pcs.position, p.birthYear, p.birthMonth
                     FROM Player_CareerStatus as pcs
                     INNER JOIN Player AS p ON pcs.mlbId = p.mlbID
                     WHERE pcs.isPrimaryPosition='1' AND pcs.serviceReached='0' AND pcs.mlbStartYear IS NOT NULL AND pcs.agedOut='0' AND pcs.ignorePlayer IS NULL''').fetchall()
for id, position, birthYear, birthMonth in tqdm(ids):
    if position == "hitting":
        years = cursor.execute("SELECT DISTINCT Year FROM Player_Hitter_MonthStats WHERE mlbId=? AND LevelId=? ORDER BY Year ASC", (id, 1)).fetchall()
    else:
        years = cursor.execute("SELECT DISTINCT Year FROM Player_Pitcher_MonthStats WHERE mlbId=? AND Level=? ORDER BY Year ASC", (id, 1)).fetchall()
    
    if len(years) == 0:
        continue
        
    # Calculate age at which player should stop accruing value
    if birthMonth < 4:
        birthYear -= 1
    stopYear = 34 + birthYear
        
    startYear = years[0][0]
    prevYear = startYear
    logged = False
    for (year,) in years:
        if (year - startYear >= 9) or (year - prevYear > 2) or year > stopYear:
            cursor.execute("INSERT INTO Player_ServiceLapse VALUES(?,?)", (id, year - 1))
            logged = True
            break
        prevYear = year
        
    # Check if the last year is not recent enough for the 2 year rule
    lastYear = years[-1][0]
    if not logged:
        if lastYear < CURRENT_SEASON - 1:
            cursor.execute("INSERT INTO Player_ServiceLapse VALUES(?,?)", (id, lastYear + 2))

cursor.execute("END TRANSACTION")
db.commit()

  0%|          | 0/3795 [00:00<?, ?it/s]

100%|██████████| 3795/3795 [00:00<00:00, 99710.49it/s]


Update signing date for players without one

In [5]:
db.rollback()
cursor = db.cursor()
cursor.execute("BEGIN TRANSACTION")

startYearData = cursor.execute('''
                               SELECT p.mlbId, pcs.careerStartYear
                               FROM Player AS p
                               INNER JOIN Player_CareerStatus as pcs ON p.mlbId = pcs.mlbId
                               WHERE pcs.isPrimaryPosition='1'
                               AND p.signingYear IS NULL
                               ''').fetchall()

for id, year in tqdm(startYearData):
    cursor.execute("UPDATE Player SET signingYear=?, signingMonth='1', signingDate='1' WHERE mlbId=?", (year, id))

cursor.execute("END TRANSACTION")
db.commit()

100%|██████████| 28396/28396 [00:00<00:00, 498182.37it/s]
